# COVID hatása New York taxiforgalmára

In [28]:
#a kód fordításához szükséges könyvtárak importálása
import pandas as pd
import glob
import os
import shapefile

# Data Model
ide jon egy diagram
## Dictionary 
oszlopok magyarázata

In [50]:
fields2 = [ 'tpep_pickup_datetime', 'tpep_dropoff_datetime','passenger_count','trip_distance','RatecodeID','PULocationID','DOLocationID','payment_type','fare_amount','extra','mta_tax','tip_amount','tolls_amount','total_amount']

# Adatok beolvasása eredeti formában

In [99]:
url="https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-03.csv"
df2018 = pd.read_csv(url, skipinitialspace=True) # itt most csak olvassuk be az egészet és majd a Data Qualitynál kiszedjük a felesleges oszlopokat
df2018.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-03-01 00:01:34,2018-03-01 00:01:43,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80
1,1,2018-03-01 00:14:34,2018-03-01 00:28:13,1,3.3,1,N,151,244,1,13.0,0.5,0.5,2.85,0.0,0.3,17.15
2,1,2018-03-01 00:51:25,2018-03-01 00:59:54,1,2.7,1,N,238,152,2,10.0,0.5,0.5,0.00,0.0,0.3,11.30
3,1,2018-03-01 00:00:01,2018-03-01 00:00:17,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80
4,1,2018-03-01 00:55:10,2018-03-01 00:56:36,1,3.7,1,N,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30


In [100]:
url="https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-03.csv"
df2020 = pd.read_csv(url, skipinitialspace=True, low_memory=False)
df2020.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-03-01 00:31:13,2020-03-01 01:01:42,1.0,4.70,1.0,N,88,255,1.0,22.0,3.0,0.5,2.00,0.0,0.3,27.80,2.5
1,2.0,2020-03-01 00:08:22,2020-03-01 00:08:49,1.0,0.00,1.0,N,193,193,2.0,2.5,0.5,0.5,0.00,0.0,0.3,3.80,0.0
2,1.0,2020-03-01 00:52:18,2020-03-01 00:59:16,1.0,1.10,1.0,N,246,90,1.0,6.0,3.0,0.5,1.95,0.0,0.3,11.75,2.5
3,2.0,2020-03-01 00:47:53,2020-03-01 00:50:57,2.0,0.87,1.0,N,151,238,1.0,5.0,0.5,0.5,1.76,0.0,0.3,10.56,2.5
4,1.0,2020-03-01 00:43:19,2020-03-01 00:58:27,0.0,4.40,1.0,N,79,261,1.0,16.5,3.0,0.5,4.05,0.0,0.3,24.35,2.5


In [67]:
print("2018-as adattábla sorainak és oszlopainak száma: {} \n".format(df2018.shape)) #sor és oszlopszám
print("2020-as adattábla sorainak és oszlopainak száma: {} \n".format(df2020.shape))

2018-as adattábla sorainak és oszlopainak száma: (9430376, 17) 

2020-as adattábla sorainak és oszlopainak száma: (3007292, 18) 



In [70]:
print("2018-as adattábla oszloptípusai:")
print("{} \n".format(df2018.dtypes))
print("2020-as adattábla oszloptípusai:")
print("{} \n".format(df2020.dtypes))

2018-as adattábla oszloptípusai:
VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
RatecodeID                 int64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtype: object 

2020-as adattábla oszloptípusai:
VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              fl

In [74]:
print("2018-as adatok: \n")
print(df2018.isnull().sum()) # null értékek ellenőrzése, hogy van-e
print("2020-as adatok: \n")
print(df2020.isnull().sum())

2018-as adatok: 

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
dtype: int64
2020-as adatok: 

VendorID                 37487
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          37487
trip_distance                0
RatecodeID               37487
store_and_fwd_flag       37487
PULocationID                 0
DOLocationID                 0
payment_type             37487
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge      

In [76]:
print("2018-as adatok:")
print(df2018.duplicated().sum())
print("2020-as adatok:")
print(df2020.duplicated().sum())

2018-as adatok: 

0
2020-as adatok: 

0


In [78]:
print("2018-as adatok:")
print(df2018.nunique()) # egyedi értékek száma oszloponként
print("\n2020-as adatok:")
print(df2020.nunique())

2018-as adatok:
VendorID                       2
tpep_pickup_datetime     2358081
tpep_dropoff_datetime    2362057
passenger_count               10
trip_distance               4528
RatecodeID                     7
store_and_fwd_flag             2
PULocationID                 262
DOLocationID                 262
payment_type                   4
fare_amount                 1894
extra                         33
mta_tax                       13
tip_amount                  3534
tolls_amount                1009
improvement_surcharge          4
total_amount               12103
dtype: int64

 2020-as adatok:
VendorID                       2
tpep_pickup_datetime     1226256
tpep_dropoff_datetime    1226685
passenger_count               10
trip_distance               3963
RatecodeID                     7
store_and_fwd_flag             2
PULocationID                 260
DOLocationID                 261
payment_type                   4
fare_amount                 6383
extra                        

In [80]:
print("2018-as adatok:")
df2018.describe()

2018-as adatok:


,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,9.430376e+06,9.430376e+06,9.430376e+06,9.430376e+06,9.430376e+06,9.430376e+06,9.430376e+06,9.430376e+06,9.430376e+06,9.430376e+06,9.430376e+06,9.430376e+06,9.430376e+06,9.430376e+06
mean,1.567604e+00,1.601390e+00,2.864808e+00,1.042197e+00,1.629755e+02,1.611061e+02,1.312569e+00,1.265681e+01,3.343034e-01,4.973344e-01,1.847916e+00,3.253465e-01,3.000036e-01,1.596526e+01
std,4.954086e-01,1.247606e+00,3.722331e+00,4.704546e-01,6.664959e+01,7.051072e+01,4.840774e-01,9.631818e+01,4.645647e-01,7.003697e-02,2.565877e+00,1.663388e+00,1.302640e+00,9.668569e+01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-3.740000e+02,-6.000000e+01,-5.000000e-01,-8.000000e+01,-3.800000e+01,-3.000000e-01,-3.748000e+02
25%,1.000000e+00,1.000000e+00,9.400000e-01,1.000000e+00,1.140000e+02,1.070000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,8.300000e+00
50%,2.000000e+00,1.000000e+00,1.600000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,9.000000e+00,0.000000e+00,5.000000e-01,1.360000e+00,0.000000e+00,3.000000e-01,1.162000e+01
75%,2.000000e+00,2.000000e+00,2.940000e+00,1.000000e+00,2.330000e+02,2.330000e+02,2.000000e+00,1.400000e+01,5.000000e-01,5.000000e-01,2.360000e+00,0.000000e+00,3.000000e-01,1.716000e+01
max,2.000000e+00,9.000000e+00,3.028000e+02,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,2.147484e+05,6.666000e+01,1.500000e+02,4.960000e+02,9.116100e+02,4.000300e+03,2.147502e+05


In [81]:
print("\n2020-as adatok:")
df2020.describe()


2020-as adatok:


,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,2.969805e+06,2.969805e+06,3.007292e+06,2.969805e+06,3.007292e+06,3.007292e+06,2.969805e+06,3.007292e+06,3.007292e+06,3.007292e+06,3.007292e+06,3.007292e+06,3.007292e+06,3.007292e+06,3.007292e+06
mean,1.667242e+00,1.467486e+00,3.116722e+00,1.051378e+00,1.634186e+02,1.609385e+02,1.267743e+00,1.269675e+01,1.246666e+00,6.580587e-01,2.138793e+00,3.041657e-01,2.975907e-01,1.884122e+01,2.241648e+00
std,4.712009e-01,1.120098e+00,2.069343e+02,7.109302e-01,6.658183e+01,7.073925e+01,4.765033e-01,3.871551e+02,2.883275e+02,2.883250e+02,2.719280e+00,1.607296e+00,3.710814e-02,6.945971e+02,7.841027e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.259000e+03,-7.000000e+00,-5.000000e-01,-4.932200e+02,-4.000000e+01,-3.000000e-01,-1.260300e+03,-2.500000e+00
25%,1.000000e+00,1.000000e+00,9.900000e-01,1.000000e+00,1.140000e+02,1.070000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.116000e+01,2.500000e+00
50%,2.000000e+00,1.000000e+00,1.670000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,9.000000e+00,5.000000e-01,5.000000e-01,1.920000e+00,0.000000e+00,3.000000e-01,1.430000e+01,2.500000e+00
75%,2.000000e+00,1.000000e+00,3.020000e+00,1.000000e+00,2.340000e+02,2.340000e+02,2.000000e+00,1.400000e+01,2.500000e+00,5.000000e-01,2.860000e+00,0.000000e+00,3.000000e-01,1.985000e+01,2.500000e+00
max,2.000000e+00,9.000000e+00,2.698037e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,6.711001e+05,5.000008e+05,5.000005e+05,8.000000e+02,6.010200e+02,3.000000e-01,1.000004e+06,2.500000e+00


## Mefigyelések
- A VendorID minimum értéke 1, maximum értéke pedig 2. Ami érthető, hiszen láttuk hogy két fajta vendor ID van a táblázatunkban.
- A Passenger_count minimum értéke 0, ez lehet valamilyen hiba is vagy esetleg egy olyan helyzet amikor a sofőr nem szállított utast.
- Három katerogikus oszlopunk van (RatecodeID, store_and_fwd_flag, payment_type). Ezeknél jól látható, hogy hány típusuk van, ott ahol az oszloponkénti egyedi értékeket vizsgáltuk.
- A két időpontot tartalmazó oszlopunk eredeti formában Object típusúak, amely számunkra nem túl hasznos, hiszen nem tudunk meg belőlük felhasználható adatokat. Így ezeket később még átalakítjuk.
- A total_amount oszlop összegzi az összes értéket azon oszlopokból, amelyek árakat tartalmaznak. Így később csak ezt az oszlopot fogjuk megtartani a vizualizációk során.

### Eltérések
ide még kell majd megfigyelés
- adattípusok eltérőek, de nincs közöttük olyan eltérés ami elrontani az adatelemzés folyamatát
- A 2020-as adattáblában jóval kevesebb sorunk van mint a 2018-asban. Ami várható volt.
- A 2020-as táblában vannak null értékek míg a 2018-asban nem. De ezen null értékeket a Data Quality check során eltávolítjuk. SZámszerűen 37.487 sort.


# Lookup table a taxi zónákhoz

In [113]:
#url="https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page" kell még a rendes url 
df_taxi_zones = pd.read_csv('C:/Users/Heni/Desktop/egyetem/7_felev/BigData/taxi+_zone_lookup.csv', skipinitialspace=True)
df_taxi_zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [114]:
df_taxi_zones.shape

(265, 4)

In [115]:
df_taxi_zones.isnull().sum()

LocationID      0
Borough         0
Zone            1
service_zone    2
dtype: int64

In [116]:
df_taxi_zones = df_taxi_zones.dropna(axis=0, how='any')

## Megjegyzés: 
Észrevettük, hogy két sor az id-n kivul ugyanolyan adatokat tárol a Borough/Zone oszlopokban, csak más id-val \
A kettő megkülönböztetése érdekében és a térképen elfoglalt helyzetük alapján szerkesztettük ezt a forrásfájlt, és a neveket a következőre változtattuk: \
- id 56 --> Corona (South)
- id 57 --> Corona (East)

Valamint észrevettük, hogy két sora tartalmaz null értéket. Ezen két sor a 264, és a 265-ös ID-jú sor, így ezeket töröltük ezen táblázatból.

# Data Quality check and data preparation

### Nan értékek törlése 
Ki akartuk szedni a táblából azon sorokat, amelyek bármely tulajdonságuknál Nan értéket tartalmaztak, azaz nem tartalmaztak értelmezhető adatot. \
Valamint, hogy megfelelően lehessen majd illeszteni a táblákat a lookup taxi zónás táblához. Leellenőriztük, hogy a táblában csak megfelelő Location ID-k szerepeljenek. Azaz töröltük azon sorokat amelyeknél vagy 264 vagy 265-s érték szerepelt bármely LocationID-s oszlopukban, hiszen ezen pozíciókhoz nincs értelmes adatunk a taxi zónás lookup táblánkban.

In [118]:
df2018 = df2018.dropna(axis=0, how='any') #itt kidobom az osszes olyan sort amiben valahol is van Nan érték
df2020 = df2020.dropna(axis=0, how='any')

In [119]:
#Nem ismert LocationID-k törlése
PU_NaN_condition = df2018[df2018['PULocationID'].isin(['264', '265'])].index
df2018 = df2018.drop(PU_NaN_condition)
DO_NaN_condition = df2018[df2018['DOLocationID'].isin(['264', '265'])].index
df2018 = df2018.drop(DO_NaN_condition)
PU_NaN_condition = df2020[df2020['PULocationID'].isin(['264', '265'])].index
df2020 = df2020.drop(PU_NaN_condition)
DO_NaN_condition = df2020[df2020['DOLocationID'].isin(['264', '265'])].index
df2020 = df2020.drop(DO_NaN_condition)

## Dátum és idő oszlop kezelése
Az eredeti táblában az utasok felvételi és kiszállási ideje Objectként van eltárolva. \
Úgy gondoltuk, hogy hasznos lenne feldarabolni ezen oszlopot, hogy plusz adatokra tehessünk szert mint például az idő és a dátum szétszedése. \
Ennek segítségével például meg tudjuk majd később különböztetni, hogy a hét mely napjáról van szó vagy ki tudjuk számolni, hogy mennyi időt ölelt fel az utazás.

In [42]:
pu_time = split_date_time(tpep_PU)
do_time = split_date_time(tpep_DO)
trip_duration = trip_duration_calc(pu_time, do_time)
pu_trip_weekday = weekday_name_identify(pu_time)
do_trip_weekday = weekday_name_identify(do_time)

#We write the new columns to df
df['PU_datetime'] = pu_time
df['DO_datetime'] = do_time
df['trip_duration'] = trip_duration
df['PU_weekday'] = pu_trip_weekday
df['DO_weekday'] = do_trip_weekday

KeyError: 1

In [40]:
print('Proportion of trips without any passenger')
ratio = df2018[df2018['passenger_count']==0].shape[0]/df2018.shape[0]*100

print("{:.2f}".format(ratio),'%')


Proportion of trips witout any passenger
0.70 %


In [ ]:
#We create two dataframes, one with only the trips with no passengers, the other with passengers
no_pass_condition = df2018[df2018['passenger_count'] == 0].index
df2018_full = df2018.drop(no_pass_condition)
df2018_empty = df2018[df2018['passenger_count']==0]

# HeatMap-hez szükséges dolgok 

In [100]:
def get_lat_lon(sf):
    content = []
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]
        
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        
        content.append((loc_id, x, y))
    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])


In [101]:
sf = shapefile.Reader("data/taxi_zones.shp")
fields_name = [field[0] for field in sf.fields[1:]]
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]

df_loc = pd.DataFrame(shp_attr).join(get_lat_lon(sf).set_index("LocationID"), on="LocationID")
df_loc.head()


,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,longitude,latitude
0,1,0.116357,0.000782,Newark Airport,1,EWR,9.366817e+05,190522.130278
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,1.033536e+06,161853.982300
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,1.027136e+06,254730.010849
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,9.904240e+05,203100.040432
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,9.321332e+05,139954.541936
